# Language Modeling on Shakespeare’s Works Using N-Grams

## Objective:
The goal of this project is to build and evaluate unigram, bigram, and trigram language models using the Complete Works of William Shakespeare. We aim to:

- Train statistical n-gram models on the text corpus.

- Compute perplexity to evaluate model performance.

- Generate sentences by sampling from these models.

## Dataset:
We use the publicly available dataset from Kaggle: [Shakespeare Online - Complete Works](https://www.kaggle.com/datasets/kewagbln/shakespeareonline)

The dataset contains a single .txt file with all of Shakespeare's plays, poems, and sonnets.

The raw corpus is tokenized and split into train and test sets for evaluation.

## Tasks Overview:
- Data Preprocessing – Load and tokenize the Shakespeare corpus.

- Train N-Gram Models – Unigram, Bigram, and Trigram with Add-1 smoothing.

- Evaluate – Compute perplexity scores on the test set.

- Generate Sentences – Use probabilistic sampling

## Import necessary libraries

In [1]:
import re
import math
import random
from collections import defaultdict, Counter

## Tokenization¶

We define a custom tokenizer simple_tokenize to convert the raw Shakespeare text into a list of tokens (words). This function:

- Converts all text to lowercase to ensure uniformity.

- Uses regular expressions `(r"\b\w+\b")` to extract word tokens while ignoring punctuation.

This basic tokenization will help us break down the dataset into individual words for training the n-gram models.

In [2]:
def simple_tokenize(text):
    return re.findall(r"\b\w+\b", text.lower())

## Print first few lines of the text

In [5]:
with open("./t8.shakespeare.txt", 'r', encoding='utf-8') as f:
    text = f.read().lower()

print(f"Corpus length: {len(text)} characters")
print(text[:5000]) # Print first 5000 characters

Corpus length: 5458199 characters
this is the 100th etext file presented by project gutenberg, and
is presented in cooperation with world library, inc., from their
library of the future and shakespeare cdroms.  project gutenberg
often releases etexts that are not placed in the public domain!!

shakespeare

*this etext has certain copyright implications you should read!*

<<this electronic version of the complete works of william
shakespeare is copyright 1990-1993 by world library, inc., and is
provided by project gutenberg etext of illinois benedictine college
with permission.  electronic and machine readable copies may be
distributed so long as such copies (1) are for your or others
personal use only, and (2) are not distributed or used
commercially.  prohibited commercial distribution includes by any
service that charges for download time or for membership.>>

*project gutenberg is proud to cooperate with the world library*
in the presentation of the complete works of william shakesp

## Load and Split the Dataset

We define the load_and_split function to:

- Open and read the raw Shakespeare text file.
- Convert the entire content to lowercase.
- Use the simple_tokenize() function to break the text into tokens.
- Split the tokens into two parts:

    - Training set (default 80%)
    - Testing set (remaining 20%)

This prepares the data for building and evaluating the language models.

In [4]:
def load_and_split(path, split_ratio=0.8):
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read().lower()

    # Limit size for memory safety (optional)
    # text = text[:500000]  # First 500K characters

    tokens = simple_tokenize(text)
    split_point = int(len(tokens) * split_ratio)
    return tokens[:split_point], tokens[split_point:]

## Define the N-Gram Language Model¶

The `NGramModel` class implements a basic statistical language model for any value of n (unigram, bigram, trigram, etc.).

Methods:

**\_\_init\_\_()**: Initializes the model with:

- `n`: the size of the n-gram (e.g., 1, 2, or 3).

- `ngram_counts`: a nested dictionary to count n-gram occurrences.

- `context_counts`: tracks total counts of each context (prefix of length n-1).

- `vocab`: a set of all unique tokens.

**train(tokens)**

- Adds `<s>` (start) and `</s>` (end) tokens to each sentence.
- Builds the n-gram and context frequency tables from the training data.

**get_prob(context, word, alpha=1.0)**

- Computes the probability of a word given its context using **Laplace (Add-1) smoothing**:

$$ P(w∣context) = \frac{count(context, w)+ \alpha}{count(context)+ \alpha \cdot V} $$

where $V$ is the vocabulary size.

**perplexity(tokens)**

- Computes the perplexity score for a given sequence of tokens using:

$$ Perplexity = e^{-\frac{1}{N}\sum_{i=1}^{N}log P(w_i | context)} $$

- Perplexity indicates how well the model predicts the test data: lower is better.

> We handle prob = 0 cases by applying a penalty (float('inf')) to avoid undefined logarithms.

In [7]:
class NGramModel:
    def __init__(self, n):
        self.n = n
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = Counter()
        self.vocab = set()

    def train(self, tokens):
        tokens = ['<s>'] * (self.n - 1) + tokens + ['</s>']
        self.vocab.update(tokens)
        for i in range(len(tokens) - self.n + 1):
            context = tuple(tokens[i:i + self.n - 1])
            word = tokens[i + self.n - 1]
            self.ngram_counts[context][word] += 1
            self.context_counts[context] += 1

    def get_prob(self, context, word, alpha=1.0):
        context = tuple(context)
        vocab_size = len(self.vocab)

        # Laplace smoothing
        count = self.ngram_counts[context][word]
        total = self.context_counts[context]
        return (count + alpha) / (total + alpha * vocab_size)


    def perplexity(self, tokens):
        tokens = ['<s>'] * (self.n - 1) + tokens + ['</s>']
        log_prob_sum = 0
        N = 0
        for i in range(len(tokens) - self.n + 1):
            context = tokens[i:i + self.n - 1]
            word = tokens[i + self.n - 1]
            prob = self.get_prob(context, word)

            # Prevent log(0)
            if prob > 0:
                log_prob_sum += -math.log(prob)
            else:
                log_prob_sum += float('inf')  # Worst case penalty

            N += 1
        return math.exp(log_prob_sum / N)

## Load Dataset and Train N-Gram Models

In this step, we:

- Load and tokenize the Shakespeare corpus using the load_and_split() function.
- Split the data into:

    - Training tokens (train_tokens)
    - Testing tokens (test_tokens)

- Initialize and train three language models:

    - Unigram (n = 1)
    - Bigram (n = 2)
    - Trigram (n = 3)

Each model learns frequency-based word patterns from the training data using the logic defined in the `NGramModel` class.

In [8]:
# Load and split
train_tokens, test_tokens = load_and_split("./t8.shakespeare.txt")

# Train models
unigram = NGramModel(1)
bigram = NGramModel(2)
trigram = NGramModel(3)

unigram.train(train_tokens)
bigram.train(train_tokens)
trigram.train(train_tokens)

In [9]:
train_tokens

['this',
 'is',
 'the',
 '100th',
 'etext',
 'file',
 'presented',
 'by',
 'project',
 'gutenberg',
 'and',
 'is',
 'presented',
 'in',
 'cooperation',
 'with',
 'world',
 'library',
 'inc',
 'from',
 'their',
 'library',
 'of',
 'the',
 'future',
 'and',
 'shakespeare',
 'cdroms',
 'project',
 'gutenberg',
 'often',
 'releases',
 'etexts',
 'that',
 'are',
 'not',
 'placed',
 'in',
 'the',
 'public',
 'domain',
 'shakespeare',
 'this',
 'etext',
 'has',
 'certain',
 'copyright',
 'implications',
 'you',
 'should',
 'read',
 'this',
 'electronic',
 'version',
 'of',
 'the',
 'complete',
 'works',
 'of',
 'william',
 'shakespeare',
 'is',
 'copyright',
 '1990',
 '1993',
 'by',
 'world',
 'library',
 'inc',
 'and',
 'is',
 'provided',
 'by',
 'project',
 'gutenberg',
 'etext',
 'of',
 'illinois',
 'benedictine',
 'college',
 'with',
 'permission',
 'electronic',
 'and',
 'machine',
 'readable',
 'copies',
 'may',
 'be',
 'distributed',
 'so',
 'long',
 'as',
 'such',
 'copies',
 '1',
 'a

## Evaluate Models Using Perplexity

We evaluate the trained models using the perplexity metric, which measures how well a language model predicts a sequence of words in the test set.

In [10]:
# Evaluate Perplexity

print("Unigram Perplexity:", unigram.perplexity(test_tokens))
print("Bigram Perplexity :", bigram.perplexity(test_tokens))
print("Trigram Perplexity:", trigram.perplexity(test_tokens))

Unigram Perplexity: 1184.4445469705927
Bigram Perplexity : 3352.331453663995
Trigram Perplexity: 14283.12273454804


## Generate Sentences via Sampling

This function uses probabilistic sampling to generate new sentences from a trained N-gram model.

**generate_sentence(model, max_len=20):**

- Initializes the sentence with `<s>` tokens (length depends on n).

- At each step:

    - Takes the current context.
    - Samples the next word based on the learned n-gram probabilities.
    - Stops when it reaches the `</s>` token or max length.
    - Returns the generated sentence (excluding the initial `<s>` tokens).

Note:
- If the context is unseen, the model samples a word randomly from the vocabulary.
- For known contexts, it performs weighted random sampling based on frequency.

This method mimics how real language is produced: word-by-word, conditioned on context.

In [11]:
def generate_sentence(model, max_len=20):
    sentence = ['<s>'] * (model.n - 1)  # Context initialization
    for _ in range(max_len):
        context = tuple(sentence[-(model.n - 1):]) if model.n > 1 else tuple()
        next_words = model.ngram_counts[context]

        # If we’ve never seen the context, sample from whole vocab
        if not next_words:
            next_word = random.choice(list(model.vocab))
        else:
            # Weighted random sampling
            words, counts = zip(*next_words.items())
            total = sum(counts)
            probs = [c / total for c in counts]
            next_word = random.choices(words, weights=probs)[0]

        if next_word == '</s>':
            break
        sentence.append(next_word)

    return ' '.join(sentence[model.n - 1:])

In [12]:
print("\n--- UNIGRAM SAMPLES ---")
for _ in range(5):
    print(generate_sentence(unigram))

print("\n--- BIGRAM SAMPLES ---")
for _ in range(5):
    print(generate_sentence(bigram))

print("\n--- TRIGRAM SAMPLES ---")
for _ in range(5):
    print(generate_sentence(trigram))


--- UNIGRAM SAMPLES ---
when wish second download hall lay me out page find her disguis her of do my fearful is prison fools
thy dies and whole only wrong overdone nails of o and to grace of swaggerer kinsmen s the suffolk venetian
other exit i me have sir dare daughter need pound an the field wherefore work isabella dear why himself d
then give darkness and the the tell senator have are on ambassador still from to hold his morn so her
have his if we madam shall good mother have promise poor to you without your to like triumph dozen to

--- BIGRAM SAMPLES ---
this a touch me to enter page that nourishes our very very cunning hides wrongs are to speak like a
this fellow to passion ay my troth not beg the rank our princes lie upon our judgment to shield thee
this resolution whereso er the murmuring lips that which i do not death and since you not he reports go
this hair dabbled in christendom shall ne er the law to his gown big for us lartius between master brook
this rest and be an hon